# Example of using ioh-explainer

First we need to define the configuration space of an algorithm. We take a standard PSO algorithm as example.

In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

from ioh_xplainer import explainer
#import pyswarms as ps
from modde import ModularDE, Parameters
import numpy as np
from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import autokeras as ak

cs = ConfigurationSpace({
    "F": (0.05, 2.0),              # Uniform float
    "CR" : (0.05, 1.0),            # Uniform float
    "lambda_": (1, 20)             # Uniform int
})

steps_dict = {
    "F": 20, 
    "CR" : 20,
    "lambda_": 10
}

def run_de(func, config, budget, dim, *args, **kwargs):
    item = {'F': np.array([float(config.get('F'))]), 'CR':np.array([float(config.get('CR'))]),  'lambda_' : int(config.get('lambda_'))*dim }
    item['budget'] = int(budget)
    c = ModularDE(func, **item)
    try:
        c.run()
        return []
    except Exception as e:
        print(f"Found target {func.state.current_best.y} target, but exception ({e}), so run failed")
        return []

de_explainer = explainer(run_de, 
                 cs , 
                 dims = [5,10,20],#,10,40],#, 10, 20, 40 
                 fids = [1,5,7,13,18,20,23], #,5
                 iids = 5, #20 
                 reps = 5, 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = steps_dict,
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)

Using TensorFlow backend


2023-08-03 18:30:37.363341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 18:30:38.543754: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
if False:
    de_explainer.run(paralell=True)
    #de_explainer.save_results("de_results.pkl")

In [3]:
de_explainer.load_results("de_results.pkl")
de_explainer._create_grid()

x = de_explainer.df[(de_explainer.df['fid'] == 1) & (de_explainer.df['dim'] == 5)][["F","CR","lambda_"]].to_numpy()

#y = de_explainer.df[(de_explainer.df['fid'] == 1) & (de_explainer.df['dim'] == 5)]["auc"].to_numpy()
#np.savetxt("sobol/x.csv", x)
#np.savetxt("sobol/y.csv", y)

#de_explainer.plot(partial_dependence=False, best_config=False)


Evaluating 4000 configurations.


In [4]:
print(de_explainer.performance_stats())

             d=5                                                           \
        Function        single-best           avg-best                avg   
0         Sphere  7228.42 (3117.22)  7421.97 (3224.87)  7616.98 (2978.30)   
1    LinearSlope   3560.97 (339.21)   3560.97 (339.21)  8815.62 (2012.77)   
2  StepEllipsoid   3528.80 (196.81)  3915.63 (1294.94)  4950.28 (1952.10)   
3     SharpRidge   3165.82 (127.76)   3165.82 (127.76)  4827.10 (2198.16)   
4  Schaffers1000   3904.31 (505.93)  4817.97 (2321.06)  5144.79 (2150.92)   
5       Schwefel    2229.93 (29.76)  3178.48 (2059.04)  4523.10 (2366.16)   
6       Katsuura  6388.20 (3018.33)  6503.68 (2933.37)  6436.83 (2923.82)   

            d=10                                                           \
        Function        single-best           avg-best                avg   
0         Sphere  5514.11 (2325.37)  6219.68 (2652.49)  7145.22 (2993.14)   
1    LinearSlope   3437.49 (250.99)   3756.54 (754.59)  7280.22 (2159.06)  

In [5]:
from IPython.display import display
#df = de_explainer.behaviour_stats()
#df.style.bar(cmap='viridis')

#do it for f1 and f5
df = de_explainer.behaviour_stats(per_fid=True)
display(df.style.bar(cmap='viridis'))

AttributeError: 'tuple' object has no attribute 'style'